In [9]:
import pandas as pd
import os
import numpy as np
import scipy

In [10]:
files = os.listdir('results_server/')
filename = "results_merged/results_2024_04_12"

In [11]:
# MERGING RESULTS
files = os.listdir('results_server/')
dfs = [
    pd.read_pickle('results_server/{}'.format(f))
    for f in files
    if f[-3:] == 'pkl'
]
df = pd.concat(dfs, ignore_index=True)
df.sort_values(by=['H']).to_csv(filename+".csv")
df.to_pickle(filename+".pkl")

In [12]:
df = pd.read_pickle(filename+".pkl")

In [13]:
df[df["H"]==0.25].sort_values(by=["res"])

,dscrt_train,dscrt,N,H,space,kappa,kappa_T,sigma,initial,constraint,...,optim_tolerance_change,optim_max_iter,max_gb,MC_,sig_comp,res,err,time,train_time,eval_time
23,100,100,4,0.25,sig,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-44.724184,0.142154,2024-03-16 01:27:09,5242.719766,21.204008
42,100,100,3,0.25,sig,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-44.378982,0.142875,2024-03-15 20:53:23,3418.838006,14.513324
29,100,100,4,0.25,log,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-37.984288,0.140608,2024-03-16 03:03:14,11428.776664,16.276386
37,100,100,3,0.25,log,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-35.701183,0.152332,2024-03-15 22:54:59,10704.653188,15.106199
26,100,100,2,0.25,log,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-30.420326,0.161839,2024-03-15 17:06:41,9498.449243,12.994428
20,100,100,2,0.25,sig,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-29.815827,0.173614,2024-03-15 16:33:06,864.115228,9.923180
61,100,100,1,0.25,log,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-1.119903,0.027528,2024-03-15 03:36:00,8957.336627,12.658896
65,100,100,1,0.25,sig,0.001,0.1,0.20,1.0,None,...,0.0,100,128,262144,tX,-0.707801,0.025423,2024-03-15 03:42:43,271.283547,7.757205
1,100,100,7,0.25,sig,0.001,0.1,0.02,1.0,None,...,0.0,100,128,262144,tX,-0.444921,0.002819,2024-03-13 20:57:25,51550.620400,97.154932
4,100,100,6,0.25,sig,0.001,0.1,0.02,1.0,None,...,0.0,100,128,262144,tX,-0.438143,0.002791,2024-03-13 08:15:37,28199.954909,53.088643


In [8]:
discrt = 100
space = ['sig', 'log']
Ns = [1, 2, 3, 4]

df_ = df.query('dscrt == @discrt and N in @Ns and space in @space')
df_ = df_.sort_values(by=['sig_comp', 'N'], ascending=[False, True])

for H in [-1, 1/4, 1/3, 1/2, 0.7, 0.9, 1.0]:
    
    if H >= 0:
        filt = df_.query("H == {}".format(H))[["H", "N", "res", "err", "space"]]
        values = filt["res"].to_numpy().round(3)
        v_min = filt["res"].dropna().to_numpy().min().round(4)
        if H == 1/3:
            print("1/3", end=' ')
        elif H == 1.0:
            print("1", end=' ')
        elif H > 1/2:
            print("{}/{}".format(int(H*10), 10), end=' ')
        else:
            print("{}/{}".format(*H.as_integer_ratio()), end = ' ')
        

    for spc in space:
        for N in Ns:
            if H == -1:
                print('& $N = {}$ '.format(N), end=' ')
            else:
                res = filt.query("N == @N and space == @spc")["res"]
                if len(res) == 0:
                    print('&      ', end=' ')
                else:
                    v = res.to_numpy()[0].round(4)
                    v_rel = -v #100*(-v - twap)/twap 
                    if v == v_min:
                        print('& \\textbf{{ {:0.3f} }}'.format(v_rel), end=' ')
                    else:
                        print('& {:0.3f}'.format(v_rel), end=' ')
    if False:
        if H == -1:
            print('& bm ', end=' ')
        elif H == 1/3:
            print('& {:0.3f} '.format(0.4144001982160647), end=' ')
        elif H == 0.7:
            print('& {:0.3f} '.format(2.1403790882061484), end=' ')
                        
    print(' \\\\ ')

& $N = 1$  & $N = 2$  & $N = 3$  & $N = 4$  & $N = 1$  & $N = 2$  & $N = 3$  & $N = 4$   \\ 
1/4 & 0.708 & 29.816 & 44.379 & \textbf{ 44.724 } & 1.120 & 30.420 & 35.701 & 37.984  \\ 
1/3 & 0.276 & 11.691 & 16.063 & \textbf{ 16.087 } & 0.471 & 11.944 & 13.142 & 13.082  \\ 
1/2 & 0.005 & 0.002 & 0.009 & \textbf{ 0.025 } & -0.016 & 0.011 & -0.040 & -0.004  \\ 
7/10 & 0.210 & 10.918 & \textbf{ 13.128 } & 12.910 & 1.048 & 11.197 & 11.826 & 11.470  \\ 
9/10 & 0.777 & 36.820 & \textbf{ 45.518 } & 44.799 & 8.810 & 39.988 & 40.322 & 40.418  \\ 
1 & 1.105 & 53.220 & 68.073 & 74.267 & 71.620 & 73.737 & 74.919 & \textbf{ 75.497 }  \\ 


In [18]:
100 * 3*10**(-7) / twap

3.0029732408325072e-05

In [19]:
bench03 = 1.0031498
bench07 = 1.0203925

100 * (bench03 - twap) / twap, 100 * (bench07 - twap) / twap

(0.4144001982160647, 2.1403790882061484)

# TWAP

In [21]:
import torch

from signature_control.examples.control.optimal_execution.run import loss_fn, rde_model, new_batch, params, update_params
from signature_control.examples.control.strategies import TimeWeightedAverage
from signature_control.examples.control.run_environment import run

In [17]:
x0 = 1.0
q0 = 1.0
T = 1.0
kappa = 0.001
kappa_T = 0.1

twap = x0*q0 - q0**2 * kappa * kappa_T / (kappa + T*kappa_T)
alpha = q0 * T * kappa_T / (kappa + T*kappa_T)
twap, alpha

(0.999009900990099, 0.9900990099009901)

In [51]:
import torch

In [56]:
a = torch.linspace(0, 1, 30).reshape(10,3)

a.mean(dim=1)

tensor([0.0345, 0.1379, 0.2414, 0.3448, 0.4483, 0.5517, 0.6552, 0.7586, 0.8621,
        0.9655])

In [3]:
import iisignature
import numpy as np

In [9]:
X = np.random.randn(1000, 100, 2)

In [11]:
iisignature.sig(X, 1).shape

(1000, 2)

In [8]:
import torch

('self',
 'params',
 'lr',
 'max_iter',
 'max_eval',
 'tolerance_grad',
 'tolerance_change',
 'history_size',
 'line_search_fn',
 'defaults')